In [13]:
#2021-07-10
import pandas as pd
import numpy as np
# AutoML을 이용한 ML 구현
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
import copy
# suboptmal problem의 iteration 문제는 무시할 것! (suboptimal의 계산횟수를 한정하여 수렴시키겠다는 얘기)
import warnings
warnings.filterwarnings('ignore')


from sklearn.neural_network import MLPRegressor
# hidden_layer_sizes = (100,) , (10, 10, ) 정도만 activation = 'relu', 'logistic'까지만, alpha =0.0001, solver = 'lbfgs', 'adam'까지만

# csv에 기록된 feature는 아래와 같음
col_name = ['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계']

# 예측을 위해 사용하는 feature들
# 일자: 월만 추출하기로 함
# 요일: 요일은 숫자로 변환하여 사용함
# 본사정원수, 본사휴가자수, 본사출장자수, 본사시간외근무명령서승인건수, 현본사소속재택근무자수
# 전날 휴무면 1로 표시
# 뒷날 휴무면 1로 표신
# 일찍 마치는 날이면 1로 표시
# lunch는 점심메뉴에 맛있다고 판단할 수 있는 반찬들이 몇개 들어갔는가?
# dinner는 저녁메뉴에 맛있다고 판단할 수 있는 반찬들이 몇개 들어갔는가?
need_col = ['일자', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
        '현본사소속재택근무자수', '전날휴무', '뒷날휴무', 'lunch', 'dinner', '중식계', '석식계', '일찍마침']

In [14]:
# 인코딩 UTF-8이 아니므로 지정하여 불러옴
df = pd.read_csv("train.csv", encoding="CP949")
dr = copy.deepcopy(df)
# 저녁 식사 인원이 거의 없는 날은 석식메뉴에 사유가 적힘
no_dinner_menu = set(df.loc[(df["석식계"]==0), "석식메뉴"])
delicious = []

In [15]:
"""
REQUIREMENTS: "일자"(0000-00-00 string)
FUNCTION: "일자"에서 월 정보와 년 정보를 칼럼으로 추가해 반환
"""
def date_to_month(given, one_hot_to_month = False, multi_coll = False):
    given['일자'] = pd.to_datetime(given['일자'])
    given['년도'] = pd.to_numeric(given['일자'].dt.year)
    given['일자'] = pd.to_numeric(given['일자'].dt.month)
    if one_hot_to_month == False:
        return given
    
    given['일자_1'] = given.apply(lambda x: 1 if x['일자']==1 else 0, axis=1)
    given['일자_2'] = given.apply(lambda x: 1 if x['일자']==2 else 0, axis=1)
    given['일자_3'] = given.apply(lambda x: 1 if x['일자']==3 else 0, axis=1)
    given['일자_4'] = given.apply(lambda x: 1 if x['일자']==4 else 0, axis=1)
    given['일자_5'] = given.apply(lambda x: 1 if x['일자']==5 else 0, axis=1)
    given['일자_6'] = given.apply(lambda x: 1 if x['일자']==6 else 0, axis=1)
    given['일자_7'] = given.apply(lambda x: 1 if x['일자']==7 else 0, axis=1)
    given['일자_8'] = given.apply(lambda x: 1 if x['일자']==8 else 0, axis=1)
    given['일자_9'] = given.apply(lambda x: 1 if x['일자']==9 else 0, axis=1)
    given['일자_10'] = given.apply(lambda x: 1 if x['일자']==10 else 0, axis=1)
    given['일자_11'] = given.apply(lambda x: 1 if x['일자']==11 else 0, axis=1)
    given['일자_12'] = given.apply(lambda x: 1 if x['일자']==12 else 0, axis=1)
     
    if multi_coll == True:
        return given[list(set(given.columns)-{"일자1"})]
    elif multi_coll == False:
        return given

In [16]:
"""
REQUIREMENTS: "요일"(월화수목금으로 표시된)
FUNCTION: "전날휴무", "뒷날휴무" column을 생성하고 휴무면 1로 표시하여 데이터프레임을 반환
"""
def prior_posterior(given):
    given["전날휴무"] = given["요일"].shift(1).fillna(0)
    given["뒷날휴무"] = given["요일"].shift(-1).fillna(0)
    
    # 전후날 근무날이면 0으로 표시하기
    given.loc[(given["요일"]=="월") & (given["뒷날휴무"]=="화"), "뒷날휴무"] = 0
    given.loc[(given["요일"]=="화") & (given["뒷날휴무"]=="수"), "뒷날휴무"] = 0
    given.loc[(given["요일"]=="수") & (given["뒷날휴무"]=="목"), "뒷날휴무"] = 0
    given.loc[(given["요일"]=="목") & (given["뒷날휴무"]=="금"), "뒷날휴무"] = 0
    given.loc[(given["요일"]=="화") & (given["전날휴무"]=="월"), "전날휴무"] = 0
    given.loc[(given["요일"]=="수") & (given["전날휴무"]=="화"), "전날휴무"] = 0
    given.loc[(given["요일"]=="목") & (given["전날휴무"]=="수"), "전날휴무"] = 0
    given.loc[(given["요일"]=="금") & (given["전날휴무"]=="목"), "전날휴무"] = 0
    given.loc[given["전날휴무"] != 0, "전날휴무"] = 1
    given.loc[given["뒷날휴무"] != 0, "뒷날휴무"] = 1

    return given

In [17]:
"""
REQUIREMENTS: "석식메뉴"
FUNCTION: 일찍 퇴근하는 날의 "석식메뉴"에는 휴무의 원인이 적혀 있음
"""
def no_work(given):
    given["일찍마침"] = given["석식메뉴"].map(lambda x: 1 if x in no_dinner_menu else 0)
    return given

In [18]:
"""
REQUIREMENTS: "중식계"와 "석식계" feature
FUNCTION: 한 주내 석식계와 중식계의 식사인원은 월요일에서 금요일로 갈 수록 감소하는데
            증가하는 날이 있다. 아마도 이런 날은 식단이 맛있을 것이므로
            그 식단의 반찬을 추출하면 맛있는 메뉴의 list를 찾을 수 있을 것이다.
"""
def find_delicious_menu(given):
    temp = []
    current_lunch = []
    current_dinner = []
    given_lid= list(given.columns).index("중식메뉴")
    given_did= list(given.columns).index("석식메뉴")
    prior_id = list(given.columns).index("전날휴무")
    posterior_id = list(given.columns).index("뒷날휴무")
    l_numid = list(given.columns).index("중식계")
    d_numid = list(given.columns).index("석식계")
    t = list(given.columns).index('본사정원수')
    f1 = list(given.columns).index('본사휴가자수')
    f2 = list(given.columns).index('본사출장자수')
    f3 = list(given.columns).index('현본사소속재택근무자수')
    early = list(given.columns).index("일찍마침")
    for i in range(1, len(given['요일'])):
        current_lunch = given.iloc[i, given_lid].split()
        current_dinner = given.iloc[i, given_did].split()
        if ((given.iloc[i, prior_id] == 0) & (given.iloc[i, posterior_id] == 0) &
            ((given.iloc[i, l_numid]/(given.iloc[i, t]-given.iloc[i,f1]-given.iloc[i,f2]-
                                     given.iloc[i,f3])) > 1.25*given.iloc[i-1, l_numid]/
             (given.iloc[i-1, t]-given.iloc[i-1,f1]-given.iloc[i-1,f2]-
                                     given.iloc[i-1,f3]) )):
            temp += current_lunch
        if ((given.iloc[i, prior_id] == 0) & (given.iloc[i, posterior_id] == 0) &
            ((given.iloc[i, d_numid]/(given.iloc[i, t]-given.iloc[i,f1]-given.iloc[i,f2]-
                                     given.iloc[i,f3])) > 1.25*given.iloc[i-1, d_numid]/
             (given.iloc[i-1, t]-given.iloc[i-1,f1]-given.iloc[i-1,f2]-
                                     given.iloc[i-1,f3]) ) &
           (given.iloc[i-1, early] != 1)):
            temp += current_dinner
            
        temp = list(set(temp))
        temp = [ x for x in temp if ":" not in x]
        temp = [ x for x in temp if "쌀밥" not in x]
        temp = [ x for x in temp if "김치" not in x]
    
    return temp
# 이제 delicious에서는 맛있었던 메뉴들로 추정되는 것들이 담김

In [19]:
"""
REQUIREMENTS: delicious(맛있는 메뉴가 들어있는 list), "요일", "중식메뉴", "석식메뉴"
FUNCTION: 중식메뉴와 석식메뉴에 delicious와 겹치는 element의 수를 "lunch"와 "dinner" column에 저장하여
            새로운 데이터프레임을 반환
"""
def is_delicious(given):
    global delicious
    now = given
    now['lunch'] = [0 for i in range(len(given["요일"]))]
    now['dinner'] = [0 for i in range(len(given["요일"]))]
    lunch_id = list(given.columns).index("중식메뉴")
    dinner_id = list(given.columns).index("석식메뉴")
    ldelicious_id = list(now.columns).index("lunch")
    ddelicious_id = list(now.columns).index("dinner")
    
    for i in range(0, len(given['요일'])):
        now.iloc[i, ldelicious_id] = len(set(given.iloc[i, lunch_id].split()) & set(delicious))
        now.iloc[i, ddelicious_id] = len(set(given.iloc[i, dinner_id].split()) & set(delicious))
    
    return now

In [20]:
# 요일을 coded variable로 나타내는 함수
def day_process(given, one_hot_to_day = False, multi_coll = False):
    if one_hot_to_day == False:
        given.loc[given["요일"]=="월", "요일"] = 0
        given.loc[given["요일"]=="화", "요일"] = 1
        given.loc[given["요일"]=="수", "요일"] = 2
        given.loc[given["요일"]=="목", "요일"] = 3
        given.loc[given["요일"]=="금", "요일"] = 4
        return given
    elif multi_coll == True:
        given = pd.get_dummies(given, columns=['요일'], drop_first=True)
        return given
    else:
        given = pd.get_dummies(given, columns=['요일'], drop_first=False)
        return given

In [21]:
"""
REQUIREMENTS: -
FUNCTION: need_col에 저장된 column만 추출함. test=True이면 "중식계"와 "석식계"를 제외하여 반환
"""
def col_selection(given, test=False):
    if test == False:
        return given[need_col]
    if test == True:
        return given[list(set(need_col)-{"중식계", "석식계"})]

In [22]:
def pre_total_preprocess(given, test = False, one_hot_to_month = False, one_hot_to_day = False,
                     multi_coll = False):
    # delicious는 함수 외부에 존재하는 빈 list임
    global delicious
    given = prior_posterior(given)
    given = no_work(given)
    if test ==  False:
        delicious = find_delicious_menu(given)
    given = is_delicious(given)
    given = day_process(given, one_hot_to_day = one_hot_to_day, multi_coll = multi_coll)
    given = col_selection(given, test=test)
    given = date_to_month(given, one_hot_to_month = one_hot_to_month, multi_coll = multi_coll)  
    return given

In [85]:

train = copy.deepcopy(df)
train = pre_total_preprocess(train, one_hot_to_month = False)

test_data = pd.read_csv("test.csv")
test_data = pre_total_preprocess(test_data, test=True, one_hot_to_month = False)

one_hot_to_month = False
if one_hot_to_month == False:
    total_column = ['년도', "일자", "전날휴무", "뒷날휴무", "일찍마침", "lunch", "dinner", "본사정원수",
      "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수",
      "현본사소속재택근무자수", "중식계", "석식계"]
    lfeature_column = ['년도', "일자", "전날휴무","뒷날휴무", "일찍마침", "lunch", "본사정원수",
      "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수",
      "현본사소속재택근무자수"]
    dfeature_column = ['년도', "일자", "전날휴무", "뒷날휴무", "일찍마침", "dinner", "본사정원수",
      "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수",
      "현본사소속재택근무자수"]
else:
    total_column = ['년도', '일자_1', '일자_2', '일자_3',
       '일자_4', '일자_5', '일자_6', '일자_7', '일자_8', '일자_9', '일자_10', '일자_11',
       '일자_12', "전날휴무", "뒷날휴무", "일찍마침", "lunch", "dinner", "본사정원수",
      "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수",
      "현본사소속재택근무자수", "중식계", "석식계"]
    lfeature_column = ['년도', '일자_1', '일자_2', '일자_3',
       '일자_4', '일자_5', '일자_6', '일자_7', '일자_8', '일자_9', '일자_10', '일자_11',
       '일자_12', "전날휴무","뒷날휴무", "일찍마침", "lunch", "본사정원수",
      "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수",
      "현본사소속재택근무자수"]
    dfeature_column = ['년도', '일자_1', '일자_2', '일자_3',
       '일자_4', '일자_5', '일자_6', '일자_7', '일자_8', '일자_9', '일자_10', '일자_11',
       '일자_12', "전날휴무", "뒷날휴무", "일찍마침", "dinner", "본사정원수",
      "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수",
      "현본사소속재택근무자수"]

In [92]:
train.columns

Index(['일자', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
       '전날휴무', '뒷날휴무', 'lunch', 'dinner', '중식계', '석식계', '일찍마침', '년도'],
      dtype='object')

In [99]:
from sklearn.decomposition import PCA
print(train[['년도', '일자', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
       '전날휴무', '뒷날휴무']].columns)
to_conversion = train[['년도', '일자', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
       '전날휴무', '뒷날휴무']]
pca_scaler = MinMaxScaler()
pca_scaler.fit(to_conversion)
pca_train = pca_scaler.transform(to_conversion)

pca = PCA()
pca.fit(pca_train)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

Index(['년도', '일자', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '전날휴무', '뒷날휴무'],
      dtype='object')
[0.33592959 0.20301034 0.19682177 0.142551   0.04313835 0.03968375
 0.01997152 0.01357699 0.00531669]
[17.15029325 13.3323436  13.12755912 11.17204196  6.145811    5.89459215
  4.18170056  3.44785626  2.15758622]


In [88]:
X1_train, X1_test, y1_train, y1_test = train_test_split(train[lfeature_column], train["중식계"], shuffle=True)
X2_train, X2_test, y2_train, y2_test = train_test_split(train[dfeature_column], train["석식계"], shuffle=True)

In [89]:
hands_on_ml1 = MLPRegressor(hidden_layer_sizes = (100, ),
                            activation = 'relu', solver = 'lbfgs', alpha = 0.0001)
hands_on_ml2 = MLPRegressor(hidden_layer_sizes = (100, ),
                            activation = 'relu', solver = 'lbfgs', alpha = 0.0001)
scaler_X1 = MinMaxScaler()
scaler_X2 = MinMaxScaler()
scaler_y1 = MinMaxScaler()
scaler_y2 = MinMaxScaler()

scaler_X1.fit(X1_train)
scaler_X2.fit(X2_train)
scaler_y1.fit(np.array(y1_train).reshape(-1,1))
scaler_y2.fit(np.array(y2_train).reshape(-1,1))

sX1tr = scaler_X1.transform(X1_train)
sX1tt = scaler_X1.transform(X1_test)
sy1tr = scaler_y1.transform(np.array(y1_train).reshape(1,-1)).reshape(-1,1)
sy1tt = scaler_y1.transform(np.array(y1_test).reshape(-1,1)).reshape(-1,1)
sX2tr = scaler_X2.transform(X2_train)
sX2tt = scaler_X2.transform(X2_test)
sy2tr = scaler_y2.transform(np.array(y2_train).reshape(-1,1)).reshape(-1,1)
sy2tt = scaler_y2.transform(np.array(y2_test).reshape(-1,1)).reshape(-1,1)

In [90]:
hands_on_ml1.fit(sX1tr, sy1tr)
hands_on_ml2.fit(sX2tr, sy2tr)

MLPRegressor(solver='lbfgs')

In [91]:
from sklearn.metrics import mean_absolute_error

pr_y1tr = scaler_y1.inverse_transform(hands_on_ml1.predict(sX1tr).reshape(-1,1)) 
pr_y1tt = scaler_y1.inverse_transform(hands_on_ml1.predict(sX1tt).reshape(-1,1))
pr_y2tr = scaler_y2.inverse_transform(hands_on_ml2.predict(sX2tr).reshape(-1,1))
pr_y2tt = scaler_y2.inverse_transform(hands_on_ml2.predict(sX2tt).reshape(-1,1))

print("중식계 training: ", mean_absolute_error(y1_train, pr_y1tr))
print("중식계 test: ", mean_absolute_error(y1_test, pr_y1tt))
print("석식계 training: ", mean_absolute_error(y2_train, pr_y2tr))
print("석식계 test: ", mean_absolute_error(y2_test, pr_y2tt))

중식계 training:  51.31784863489709
중식계 test:  67.4901543574564
석식계 training:  33.43284047640583
석식계 test:  45.105337878951914


In [28]:
pipe = Pipeline([('preprocessing', None), ('regressor', MLPRegressor())])
pre_list = [MinMaxScaler()]
hyperparam_grid = [
    {"regressor": [MLPRegressor()], 'preprocessing': pre_list,
    "regressor__hidden_layer_sizes": [(100,)],
    "regressor__activation": ["relu"],
    "regressor__solver": ["lbfgs"],
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1]}

]
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

grid1 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)
grid2 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)

In [29]:
grid1.fit(X1_train, y1_train)
print(grid1.best_estimator_)
print(grid1.best_params_)
print(grid1.best_score_)
print(grid1.score(X1_test, y1_test))

Pipeline(steps=[('preprocessing', MinMaxScaler()),
                ('regressor', MLPRegressor(alpha=0.1, solver='lbfgs'))])
{'preprocessing': MinMaxScaler(), 'regressor': MLPRegressor(alpha=0.1, solver='lbfgs'), 'regressor__activation': 'relu', 'regressor__alpha': 0.1, 'regressor__hidden_layer_sizes': (100,), 'regressor__solver': 'lbfgs'}
-70.65777706384944
-67.45982445701011


In [30]:
grid2.fit(X2_train, y2_train)
print(grid2.best_estimator_)
print(grid2.best_params_)
print(grid2.best_score_)
print(grid2.score(X2_test, y2_test))

Pipeline(steps=[('preprocessing', MinMaxScaler()),
                ('regressor', MLPRegressor(alpha=0.001, solver='lbfgs'))])
{'preprocessing': MinMaxScaler(), 'regressor': MLPRegressor(alpha=0.001, solver='lbfgs'), 'regressor__activation': 'relu', 'regressor__alpha': 0.001, 'regressor__hidden_layer_sizes': (100,), 'regressor__solver': 'lbfgs'}
-47.19837081827621
-49.56533612649797


In [ ]:
# 지금까지 알아낸 사실 각 요일별 식사 인원은 회사에 잔존하는 인원 수보다 많을 수 없다.
# 또한, 음수도 될 수 없다.
# 그러면 엄청 큰 값은 최대치(혹은 75% qunatile)로, 엄청 작은 값은 최소치(25% quantile)로 대응시키는 게 좋을까?
# 또한 각 요일별 식사 인원의 += 3 sigma보다 많이 벗어나는 일도 기적 같은 일이다.
# 차라리 각 분류별 median 모형을 사용하는 것이 더 적합할까?
# 전날 휴일인 게 필요할까?
# 각 요일별 평균치를 대입할까?
# 각 

In [ ]:
a = pd.DataFrame(reg3.predict(test_data[lfeature_column]))

In [ ]:
b = pd.DataFrame(reg4.predict(test_data[dfeature_column]))

In [ ]:
save = pd.read_csv("sample_submission.csv")
save["중식계"] = a
save["석식계"] = b
save.columns = ["일자", "중식계", "석식계"]
save.to_csv("submit.csv", encoding="CP949", index=False)